## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

Ultimately, we want to answer the following questions. How much does the lane line curve? Am I still driving in the center of the road? 


# Camera Calibration
Each lens has its unique lens distortion based on its parameters. The monst common distortions include radial distortion and tangential distortion. Since we will be using camera to track lane lines, as well as determine curvature of the road, it becomes critical for us to remove these optical distortions. Based on the documentiaton from OpenCV (https://docs.opencv.org/3.1.0/dc/dbb/tutorial_py_calibration.html) The radial distortion can be modeled with the following function:<br />
$$x_{rad distort} = x(1+k_1r^2+k_2r^4+k_3r^6)$$ <br />
$$y_{rad distort} = y(1+k_1r^2+k_2r^4+k_3r^6)$$

and the tangential distortion is modeled as the following: <br />
$$x_{tan distort} = x+[2p_1xy+p_2(r^2+2x^2)]$$ <br />
$$y_{tan distort} = y+[p_1(r^2+2y^2)+2p_2xy]$$

Therefore, we can define the distortion coefficients $d$ as: <br />
$$d = [k_1, k_2, p_1, p_2, k_3]$$

The camera matrix is defined as:
$$\mathbf{C} = \left[\begin{array}
{rrr}
f_x & 2 & c_x \\
0 & f_y & c_y \\
0 & 0 & 1
\end{array}\right],
$$
where $f_x, f_y$ is the focal length of the camera lens and $c_x, c_y$ is the optical center. To optain camera matrix $\mathbf{C}$ and distortion coefficient $d$, we can utilize cv2.calibrateCamera( ) function from OpenCV.


In [7]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.
d = 0
# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

def camera_calibration_matrix(objpoints, imgpoints, img):
    #This function takes chessboard images with known dimensions and obtain camera calibration matrix
    localImg = np.copy(img)
    grayImg = cv2.cvtColor(localImg,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tves = cv2.calibrateCamera(objpoints, imgpoints, grayImg.shape[::-1],None,None)    
    return mtx,dist


# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    grayImg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(grayImg, (9,6),None)
    
    
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        
        mtx,dist = camera_calibration_matrix(objpoints, imgpoints, img)
        
        # Undistort images
        undistortedImg = cv2.undistort(img,mtx,dist, None, mtx)
        # Draw and display the corners
        originalImg_with_chessboard = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        undistortedImg_with_chessboard = cv2.drawChessboardCorners(undistortedImg, (9,6), corners, ret)
        
        outputImgPath = "./calibrated_images/"
        cv2.imwrite(outputImgPath+"original_%d.jpg"%d,originalImg_with_chessboard)
        cv2.imwrite(outputImgPath+"undistorted_%d.jpg"%d,undistortedImg_with_chessboard)
        d+=1

np.savetxt("camera_calibration_matrix.csv",mtx, delimiter=",")
np.savetxt("camera_distortion_coefficients.csv",dist, delimiter=",")

# Gradient and Color Threshold
Once we obtain the camera matrix and distorntion coeffcients, we will use it for distortion correction for lane line images. Instead of using simple canny edge detector, we will try to combine color and gradient threshold to get the best result. 

## Sobel Opeartor for gradient measurements
The Sobel Opeartor are used to perform convolution on the original image to determine how gradient changes. Intuitvely, we want to measure the change of gradient with respect to both x axis and y axis, as well as direction of gradient. 

In this project, I use $3 \times 3$ Scharr filters (set ksize = -1) kernels, namely $\mathbf{S_x}$ and $\mathbf{S_y}$ for the two axises. <br />
Gradient along x axis:
$$\mathbf{S_x} = \left[\begin{array}
{rrr}
-3 & 0& 3\\
-10 & 0 & 10\\
3 & 0 & 3
\end{array}\right],
$$
Gradient along y axis:
$$\mathbf{S_y} = \left[\begin{array}
{rrr}
-3 & -10 & -3 \\
0 & 0 & 0 \\
3 & 10 & 3
\end{array}\right].
$$

The magnitude $\mathbf{S}$ and direction $\theta$ of the gradient can be easily obtained through trigonometry:<br />

$$\mathbf{S} = \sqrt{\mathbf{S_x}^2+\mathbf{S_y}^2}$$<br />
$$\theta = atan(\frac{\mathbf{S_y}}{\mathbf{S_x}})$$


In [39]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import glob

d = 0 #image naming counter

def abs_sobel_thresh(img, orient='x', sobel_kernel=-1, thresh=(60, 150)):
    # Calculate directional gradient
    # Apply threshold
    thresh_min = thresh[0]
    thresh_max = thresh[1]
    grayImg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    if orient =='x':
        sobelDir = cv2.Sobel(grayImg, cv2.CV_64F, 1,0, ksize=sobel_kernel) #x oritentaion
    elif orient =='y':
        sobelDir = cv2.Sobel(grayImg, cv2.CV_64F, 0,1, ksize=sobel_kernel) #y orientation

    absSobelDir = np.absolute(sobelDir)
    scaled_sobel = np.uint8(255*absSobelDir/np.max(sobelDir))
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    return grad_binary

def mag_thresh(image, sobel_kernel=-1, mag_thresh=(50, 150)):
    # Calculate gradient magnitude
    # Apply threshold
    thresh_min = mag_thresh[0]
    thresh_max = mag_thresh[1]
    grayImg = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(grayImg, cv2.CV_64F, 1,0,ksize=sobel_kernel) #x oritentaion
    sobely = cv2.Sobel(grayImg, cv2.CV_64F, 0,1,ksize=sobel_kernel) #y orientation
    absSobelxy = np.sqrt(sobelx**2+sobely**2)
    scaled_sobel = np.uint8(255*absSobelxy/np.max(absSobelxy))
    mag_binary = np.zeros_like(scaled_sobel)
    mag_binary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    return mag_binary

def dir_threshold(image, sobel_kernel=-1, thresh=(0.5, np.pi/2)):
    # Calculate gradient direction
    # Apply threshold
    thresh_min = thresh[0]
    thresh_max = thresh[1]
    grayImg = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(grayImg, cv2.CV_64F, 1,0,ksize=sobel_kernel) #x oritentaion
    sobely = cv2.Sobel(grayImg, cv2.CV_64F, 0,1,ksize=sobel_kernel) #y orientation
    gradientDirectionImg = np.arctan2(np.absolute(sobely), np.absolute(sobelx));
    dir_binary = np.zeros_like(gradientDirectionImg)
    dir_binary[(gradientDirectionImg >= thresh_min) & (gradientDirectionImg <= thresh_max)] = 1
    return dir_binary

images = glob.glob('../test_images/test*.jpg')
for fname in images:
    image =  mpimg.imread(fname)
    ksize = -1 #3x3 scharr filter
    # Calculate gradients
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(50, 150))
    grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(0, 30))
    mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(30, 150))
    dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(0.8, np.pi/2))
    
    #Combien Result
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    outputImgPath = "./combined_sobel_images/"
    mpimg.imsave(outputImgPath+"combined_sobel_images%d.jpg"%d,combined,cmap='gray')
    d+=1
    


## Color Thresholding

Another thresoholding techqniue is used in color space. Intuitively, we want to extract the color of interest from the image that resembles the lane line (In this case, yellow and white colors) The standard RGB color space is a three dimensional vector space with Red, Green and Blue for each axis. In theory, we can directly perform color thresholding in RGB color space. However, the surrounding light can change dramatically in real-life situation, which can lead to poor performance and various of other issues. Alternatively, we can represent an image in Hue, Saturation and Value (HSV) color space or Hue, Lightness and Saturation (HLS) color space. Why do we want to perform color thresholding in those color spaces? Well, the use of Hue and Saturation are critical because they are indepentdent of brightness.

For the project, I decide to use HLS color space for color thresholding. To convert the image from RGB to HLS, I use the OpenCV function cv2.cvtColor(im, cv2.COLOR_RGB2HLS). After some testings, the saturation channel (S channel) performs the best in terms of extracting lane line, but I will combine it with Hue thresholding to get more degree of freedom. The following code demonstrate how a binary image is generated through S and H channel thresholding. 

In [45]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob

def color_threshold(imgage, S_thresh=(0, 255),H_thresh=(0, 255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    S = hls[:,:,2]
    binary_output = np.zeros_like(S)
    binary_output[(S > S_thresh[0]) & (S <= S_thresh[1])& (H > H_thresh[0])& (H <= H_thresh[1])] = 1
    return binary_output

d = 0 #image naming counter
images = glob.glob('../test_images/test*.jpg')
for fname in images:
    image =  mpimg.imread(fname)
    sat_binary = saturation_threshold(image, S_thresh=(90, 190),H_thresh=(5, 100))
    outputImgPath = "./saturation_thresh_images/"
    mpimg.imsave(outputImgPath+"saturation_thresh_images%d.jpg"%d,sat_binary,cmap='gray')
    d+=1
    


Now we can combine all thresholding techniques together

In [40]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob


# Threshold Parameters
abs_sobel_thresh_param_x = (50,150)
abs_sobel_thresh_param_y = (0,25)
mag_thresh_param = (60, 150)
dir_thresh_param = (0.9, np.pi/2)
saturation_thresh_param = (90,255)
hue_thresh_param = (10, 80)
ksize = -1 


def abs_sobel_thresh(grayImg , orient ='x', sobel_kernel=-1, thresh=(50, 150)):
    # Calculate directional gradient
    # Apply threshold
    thresh_min = thresh[0]
    thresh_max = thresh[1]

    if orient =='x':
        sobelDir = cv2.Sobel(grayImg, cv2.CV_64F, 1,0, ksize=sobel_kernel) #x oritentaion
    elif orient =='y':
        sobelDir = cv2.Sobel(grayImg, cv2.CV_64F, 0,1, ksize=sobel_kernel) #y orientation

    absSobelDir = np.absolute(sobelDir)
    scaled_sobel = np.uint8(255*absSobelDir/np.max(sobelDir))
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    return grad_binary

def mag_thresh(grayImg , sobel_kernel=-1, mag_thresh=(90, 150)):
    # Calculate gradient magnitude
    # Apply threshold
    thresh_min = mag_thresh[0]
    thresh_max = mag_thresh[1]
    sobelx = cv2.Sobel(grayImg, cv2.CV_64F, 1,0,ksize=sobel_kernel) #x oritentaion
    sobely = cv2.Sobel(grayImg, cv2.CV_64F, 0,1,ksize=sobel_kernel) #y orientation
    absSobelxy = np.sqrt(sobelx**2+sobely**2)
    scaled_sobel = np.uint8(255*absSobelxy/np.max(absSobelxy))
    mag_binary = np.zeros_like(scaled_sobel)
    mag_binary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    return mag_binary

def dir_threshold(grayImg , sobel_kernel=-1, thresh=(0.9, np.pi/2)):
    # Calculate gradient direction
    # Apply threshold
    thresh_min = thresh[0]
    thresh_max = thresh[1]
    sobelx = cv2.Sobel(grayImg, cv2.CV_64F, 1,0,ksize=sobel_kernel) #x oritentaion
    sobely = cv2.Sobel(grayImg, cv2.CV_64F, 0,1,ksize=sobel_kernel) #y orientation
    gradientDirectionImg = np.arctan2(np.absolute(sobely), np.absolute(sobelx));
    dir_binary = np.zeros_like(gradientDirectionImg)
    dir_binary[(gradientDirectionImg >= thresh_min) & (gradientDirectionImg <= thresh_max)] = 1
    return dir_binary

def color_threshold(image, S_thresh=(90, 255),H_thresh=(10, 80)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    S = hls[:,:,2]
    color_binary = np.zeros_like(S)
    color_binary[(S >= S_thresh[0]) & (S <= S_thresh[1])& (H >= H_thresh[0])& (H <= H_thresh[1])] = 1
    return color_binary


def threshold(image,ksize, abs_sobel_thresh_param_x, abs_sobel_thresh_param_y ,mag_thresh_param, dir_thresh_param,saturation_thresh_param, hue_thresh_param):
    # Convert original RGB image to Gray Image
    grayImg = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Gradient Threshold
    gradx = abs_sobel_thresh(grayImg, orient='x', sobel_kernel=ksize, thresh=abs_sobel_thresh_param_x)
    grady = abs_sobel_thresh(grayImg, orient='y', sobel_kernel=ksize, thresh=abs_sobel_thresh_param_y)
    mag_binary = mag_thresh(grayImg, sobel_kernel=ksize, mag_thresh=mag_thresh_param)
    dir_binary = dir_threshold(grayImg, sobel_kernel=ksize, thresh=dir_thresh_param)
    combined_gradient_binary = np.zeros_like(dir_binary)
    combined_gradient_binary[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    # Color Threshold
    color_binary = color_threshold(image, S_thresh=saturation_thresh_param,H_thresh=hue_thresh_param)

    # Combine Gradient Threshold and Color Threshold
    image_after_threshold_binary = np.zeros_like(color_binary)
    image_after_threshold_binary[(combined_gradient_binary == 1)|(color_binary == 1)]=1
    return image_after_threshold_binary

def mask_image(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

# Test threshold function:
#d = 0 #image naming counter
#images = glob.glob('../test_images/test*.jpg')
#for fname in images:
#    image =  mpimg.imread(fname)
#    height, width, chanels = image.shape
#    mask_vertices = np.array([[(width*0.1,height),(width/2.3,height/1.6),(width/1.7,height/1.6),(width*0.9,height)]],dtype=np.int32)
    
#    image_after_threshold_binary = threshold(image,ksize, abs_sobel_thresh_param_x,abs_sobel_thresh_param_y, mag_thresh_param, dir_thresh_param, saturation_thresh_param, hue_thresh_param)
    
#    maksed_image = mask_image(image_after_threshold_binary, mask_vertices)
#    outputImgPath = "./thresholded_images/"
#    mpimg.imsave(outputImgPath+"thresholded_images%d.jpg"%d,maksed_image,cmap='gray')
#    d+=1
    
    




# Perspective Transform
To get the curvature of the road, first we need fit the lane line with a polynomial function. While this process might seem to be straight forward, we need to deal with the distortion based on the mounted postion of our camera (In this case, the camera is mounted on the hood). To accurately fit the lane line, we need to perform a perspective transform, such that the resulting image becomes a top-down view. 


In [41]:
import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob

d = 1 #image naming counter
images = glob.glob('../test_images/test*.jpg')

def perspective_transform(image):
    if len(image.shape) > 2:
        height, width, chanels = image.shape
    else:
        height, width = image.shape
            
    src = np.array([[(width*0.1,height*0.9),
                    (width/2.3,height/1.6),
                    (width/1.7,height/1.6),
                    (width*0.9,height*0.9)]],dtype=np.float32)
    
    dst = np.array([[(0.2*width,height),
                    (0.2*width,0),
                    (0.8*width,0),
                    (0.8*width,height)]],dtype=np.float32)
    transform_matrix = cv2.getPerspectiveTransform(src,dst)
    top_down_image = cv2.warpPerspective(image, transform_matrix, (width,height), flags = cv2.INTER_LINEAR)
    return top_down_image, transform_matrix
    

#for fname in images:
#    image =  mpimg.imread(fname)
#    top_down_image, transform_matrix = perspective_transform(image)
#    outputImgPath = "./top_down_images/"
#    mpimg.imsave(outputImgPath+"top_down_images%d.jpg"%d,top_down_image,cmap='gray')
#    d+=1
    



# Lane Lines Fitting
The next step is to fit lane line using a polynomial as the following:
$$y = ax^2+bx+c$$

Before fitting the lane line, we need to process raw images with functions that we have defined above and output thresholded, masked, transformed binary images. We name these "warped_images".

In [5]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import glob

# Threshold Parameters
abs_sobel_thresh_param_x = (50,150)
abs_sobel_thresh_param_y = (0,25)
mag_thresh_param = (60, 150)
dir_thresh_param = (0.9, np.pi/2)
saturation_thresh_param = (90,255)
hue_thresh_param = (10, 80)
ksize = -1 
#mask_vertices = np.array([[(width*0.1,height*0.9),(width/2.3,height/1.6),(width/1.7,height/1.6),(width*0.9,height*0.9)]],dtype=np.int32)


# Read in images
#d = 1
#images = glob.glob('../test_images/test*.jpg')

# Loop Through images 
#for fname in images:
#    image =  mpimg.imread(fname)
#    height, width, chanels = image.shape
#    image_after_threshold_binary = threshold(image,ksize, abs_sobel_thresh_param_x,abs_sobel_thresh_param_y, mag_thresh_param, dir_thresh_param, saturation_thresh_param, hue_thresh_param)
#    maksed_image_binary = mask_image(image_after_threshold_binary, mask_vertices)
#    top_down_image = perspective_transform(maksed_image_binary)
#    outputImgPath = "./images_for_lane_lines_fitting/"
#    mpimg.imsave(outputImgPath+"warped_images%d.jpg"%d,top_down_image,cmap='gray')
#    d += 1




Now we have created warped_images for testing. I am using a slidng window apporach to fit the lane lines. For each lane line, we will have n windows each (In total, we have $2\times n$ windows for the entire image). To determine the starting point of left lane line and right lane line, we will sum the number of pixels vertically along bottom of the image. The histogram will 

In [42]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import glob
%matplotlib qt


def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    #binary_warped = binary_warped[:,:,0] #Just get the one of the three channels
    
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)


    # Create an output image to draw on and visualize the result
    out_img = (np.dstack((binary_warped, binary_warped, binary_warped))).astype(np.uint8)

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass
    

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)


    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    #plt.imshow(out_img)

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')

    return left_fit, right_fit, ploty, leftx, lefty, rightx, righty, left_fitx, right_fitx

#images = glob.glob('./images_for_lane_lines_fitting/warped_images*.jpg')
#d=0
#for fname in images:
#    test_warped_image =  mpimg.imread(fname)
#    sliding_window_img, left_fit, right_fit = fit_polynomial(test_warped_image)
#    outputImgPath = "./sliding_window_images/"
#    mpimg.imsave(outputImgPath+"sliding_window_images%d.jpg"%d,sliding_window_img)
#    d += 1

#test_warped_image = mpimg.imread('./images_for_lane_lines_fitting/warped_images5.jpg')
#sliding_window_img, left_fit, right_fit = fit_polynomial(test_warped_image)


To speed up process of finding lane line on the next frame,we utlizes the prior knowdge of where lane line could possibly be located

In [90]:
def fitpolynomial_with_previous_fitting_value(binary_warped, left_fit_before, right_fit_before,ploty):
    margin = 120
    left_fit_current, right_fit_current = (None, None)
    img_shape = binary_warped.shape
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_lane_inds = ((nonzerox > (left_fit_before[0]*(nonzeroy**2) + left_fit_before[1]*nonzeroy + 
                    left_fit_before[2] - margin)) & (nonzerox < (left_fit_before[0]*(nonzeroy**2) + 
                    left_fit_before[1]*nonzeroy + left_fit_before[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit_before[0]*(nonzeroy**2) + right_fit_before[1]*nonzeroy + 
                    right_fit_before[2] - margin)) & (nonzerox < (right_fit_before[0]*(nonzeroy**2) + 
                    right_fit_before[1]*nonzeroy + right_fit_before[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    left_fit_current = np.polyfit(lefty, leftx, 2)
    right_fit_current = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty_current = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx_current = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx_current = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fit_current, right_fit_current, ploty, leftx, lefty, rightx, righty, left_fitx_current, right_fitx_current
    

    
    
    


# Radius Curvature Calculation

The radius curvature formula is the following: 
$$R_{curve} = \frac{(1+(2ay+b)^2)^{\frac{3}{2}}}{|2a|}$$

In [44]:
import numpy as np

def measure_curvature_pixels(left_fit, right_fit, ploty, leftx, lefty, rightx, righty):
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)

    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

def measured_curvature_meters(left_fit, right_fit, ploty, leftx, lefty, rightx, righty):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''  
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    leftx = leftx[::-1]   # Reverse to match top-to-bottom in y
    rightx = rightx[::-1]  # Reverse to match top-to-bottom in y
    
    # Determine center position, assuming image 1280x720
    leftx_mid = left_fit[0]*720**2 + left_fit[1]*720 + left_fit[2]
    rightx_mid = right_fit[0]*720**2 + right_fit[1]*720 + right_fit[2]
    center_offset_meter = abs((640-(rightx_mid+leftx_mid)/2)*xm_per_pix)
    
    y_eval = np.max(ploty)
    
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad,center_offset_meter


In [45]:
import numpy
import cv2

def draw_line(undistortedImg, transform_matrix, top_down_image, ploty, left_fitx, right_fitx, left_curverad_meter,right_curverad_meter, center_offset_meter):
    warp_zero = np.zeros_like(top_down_image).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    Minv = np.linalg.inv(transform_matrix)
    
    # Generate Points
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undistortedImg.shape[1], undistortedImg.shape[0])) 
    result_image = cv2.addWeighted(undistortedImg, 1, newwarp, 0.3, 0)

    # Display Informtiaon on Image
    cv2.putText(result_image, 'Left Lane Curve: '+str("{0:.2f}".format(left_curverad_meter)) + ' meter', (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
    cv2.putText(result_image, 'Right Lane Curve: '+str("{0:.2f}".format(right_curverad_meter)) + ' meter', (100,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
    cv2.putText(result_image, 'Center Offset: '+str("{0:.2f}".format(center_offset_meter)) + ' meter', (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)

    return result_image

# Video Pipe Line Testing
Now let us test out the lane line detection algorithm on real videos

In [97]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
from numpy import genfromtxt
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob

# Load Test Image
pipe_image_example = mpimg.imread('../test_images/test1.jpg')
height, width, channel = pipe_image_example.shape
# Load Camera Calibration Matrix and Distortion Coefficients
mtx = genfromtxt('camera_calibration_matrix.csv',delimiter=',',encoding="utf8")
dist = genfromtxt('camera_distortion_coefficients.csv',delimiter=',',encoding="utf8")


# Threshold Parameters
abs_sobel_thresh_param_x = (50,150)
abs_sobel_thresh_param_y = (0,25)
mag_thresh_param = (60, 150)
dir_thresh_param = (0.9, np.pi/2)
saturation_thresh_param = (90,255)
hue_thresh_param = (10, 80)
ksize = -1 
mask_vertices = np.array([[(width*0.1,height*0.9),(width/2.3,height/1.6),(width/1.7,height/1.6),(width*0.9,height*0.9)]],dtype=np.int32)
left_fit = np.array([], dtype=np.float64)
right_fit = np.array([], dtype=np.float64)
ploty = np.array([], dtype=np.float64)

def pipeline(image):
    #global left_fit
    #global right_fit
    #global ploty
    # Acquire image shape
    height, width, chanels = image.shape
    # Undistort Image
    undistortedImg = cv2.undistort(image,mtx,dist, None, mtx)
    # Apply Threshold to remove unwanted background pixels
    image_after_threshold_binary = threshold(undistortedImg,ksize, abs_sobel_thresh_param_x,abs_sobel_thresh_param_y, mag_thresh_param, dir_thresh_param, saturation_thresh_param, hue_thresh_param)
    # Masking to get region of interst
    maksed_image_binary = mask_image(image_after_threshold_binary, mask_vertices)

    # Perspective Transform to top down view
    top_down_image, transform_matrix = perspective_transform(maksed_image_binary)
    # Use sliding window technique to acquire polynomial coefficent for line fitting
    #if left_fit
    
    
    #if left_fit.size == 0 | right_fit.size == 0: #Check if the fitting value is empyt. Perform full image sliding window if its True
    #    left_fit, right_fit, ploty, leftx, lefty, rightx, righty, left_fitx, right_fitx = fit_polynomial(top_down_image)
    #else:
    #    left_fit, right_fit, ploty, leftx, lefty, rightx, righty, left_fitx, right_fitx = fitpolynomial_with_previous_fitting_value(top_down_image, left_fit, right_fit, ploty)
    
    # For challenge videos, I need to use full image sliding window apparoch
    left_fit, right_fit, ploty, leftx, lefty, rightx, righty, left_fitx, right_fitx = fit_polynomial(top_down_image)
   
    left_curverad_meter, right_curverad_meter, center_offset_meter = measured_curvature_meters(left_fit, right_fit, ploty, leftx, lefty, rightx, righty)
    result_image = draw_line(undistortedImg, transform_matrix, top_down_image, ploty, left_fitx, right_fitx,left_curverad_meter,right_curverad_meter, center_offset_meter)
    return result_image

#result_image = pipeline(pipe_image_example)
#plt.imshow(result_image)

# Read in images
#d = 1
#images = glob.glob('../test_images/test*.jpg')

# Loop Through images 
#for fname in images:
#    image =  mpimg.imread(fname)
#    result_image = pipeline(image)
#    outputImgPath = "../output_images"
#    mpimg.imsave(outputImgPath+"/result_test_images%d.jpg"%d,result_image)
#    d += 1


In [98]:
# Clear out any stored values before feed in video stream
left_fit = np.array([], dtype=np.float64)
right_fit = np.array([], dtype=np.float64)
ploty = np.array([], dtype=np.float64)

In [93]:
test_video_output = '../output_video/project_output.mp4'
project_video = VideoFileClip('../project_video.mp4')
processed_video = project_video.fl_image(pipeline)
%time processed_video.write_videofile(test_video_output, audio = False, verbose=False)











                                                                 








t:   0%|          | 1/1260 [02:54<61:02:33, 174.55s/it, now=None]




t:   0%|          | 1/1260 [04:52<102:26:20, 292.92s/it, now=None]



t:   0%|          | 1/1260 [01:35<33:33:32, 95.96s/it, now=None]

t:   0%|          | 1/1260 [03:47<79:24:13, 227.05s/it, now=None]








                                                                  








t:   0%|          | 1/1260 [02:54<61:02:51, 174.56s/it, now=None]




t:   0%|          | 1/1260 [04:52<102:26:38, 292.93s/it, now=None]



t:   0%|          | 1/1260 [01:35<33:33:49, 95.97s/it, now=None]

t:   0%|          | 1/1260 [03:47<79:24:29, 227.06s/it, now=None]





t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]





t:   0%|          | 2/1260 [00:00<01:11, 17.57it/s, now=None]

Moviepy - Building video ../output_video/project_output.mp4.
Moviepy - Writing video ../output_video/project_output.mp4









t:   0%|          | 3/1260 [00:00<01:51, 11.23it/s, now=None]





t:   0%|          | 5/1260 [00:00<01:53, 11.04it/s, now=None]





t:   1%|          | 7/1260 [00:00<01:53, 11.00it/s, now=None]





t:   1%|          | 9/1260 [00:00<01:56, 10.73it/s, now=None]





t:   1%|          | 10/1260 [00:00<01:59, 10.50it/s, now=None]





t:   1%|          | 12/1260 [00:01<01:59, 10.43it/s, now=None]





t:   1%|          | 14/1260 [00:01<01:59, 10.41it/s, now=None]





t:   1%|▏         | 16/1260 [00:01<02:00, 10.35it/s, now=None]





t:   1%|▏         | 18/1260 [00:01<01:59, 10.39it/s, now=None]





t:   2%|▏         | 20/1260 [00:01<01:58, 10.43it/s, now=None]





t:   2%|▏         | 22/1260 [00:02<01:56, 10.64it/s, now=None]





t:   2%|▏         | 24/1260 [00:02<01:58, 10.42it/s, now=None]





t:   2%|▏         | 26/1260 [00:02<02:01, 10.17it/s, now=None]





t:   2%|▏         | 28/1260 [00:02<02:01, 10.17it/s, now=None]





t:   2%|▏         | 30/1260 [00:02<02:02, 10.0

Moviepy - Done !
Moviepy - video ready ../output_video/project_output.mp4
CPU times: user 5min, sys: 2min 34s, total: 7min 34s
Wall time: 2min 1s


Now let us try the harder video

In [99]:
test_video_output = '../output_video/challenge_video_output_fullImageSlidingWindows.mp4'
project_video = VideoFileClip('../challenge_video.mp4')
processed_video = project_video.fl_image(pipeline)
%time processed_video.write_videofile(test_video_output, audio = False, verbose=False)










                                                                    








t:   0%|          | 1/1260 [55:42<1168:57:48, 3342.55s/it, now=None]




t:   0%|          | 1/1260 [57:40<1210:21:35, 3460.92s/it, now=None]



t:   0%|          | 1/1260 [54:23<1141:28:47, 3263.96s/it, now=None]

t:   0%|          | 1/1260 [56:35<1187:19:28, 3395.05s/it, now=None]








                                                                    








t:   0%|          | 1/1260 [55:42<1168:58:06, 3342.56s/it, now=None]




t:   0%|          | 1/1260 [57:40<1210:21:53, 3460.93s/it, now=None]



t:   0%|          | 1/1260 [54:23<1141:29:04, 3263.97s/it, now=None]

t:   0%|          | 1/1260 [56:35<1187:19:44, 3395.06s/it, now=None]





t:   0%|          | 0/485 [00:00<?, ?it/s, now=None]





t:   0%|          | 2/485 [00:00<00:24, 19.41it/s, now=None]

Moviepy - Building video ../output_video/challenge_video_output_fullImageSlidingWindows.mp4.
Moviepy - Writing video ../output_video/challenge_video_output_fullImageSlidingWindows.mp4









t:   1%|          | 3/485 [00:00<00:36, 13.31it/s, now=None]





t:   1%|          | 5/485 [00:00<00:38, 12.36it/s, now=None]





t:   1%|▏         | 7/485 [00:00<00:40, 11.82it/s, now=None]





t:   2%|▏         | 9/485 [00:00<00:41, 11.48it/s, now=None]





t:   2%|▏         | 11/485 [00:00<00:42, 11.21it/s, now=None]





t:   3%|▎         | 13/485 [00:01<00:43, 10.81it/s, now=None]





t:   3%|▎         | 15/485 [00:01<00:44, 10.49it/s, now=None]





t:   3%|▎         | 16/485 [00:01<00:46, 10.14it/s, now=None]





t:   4%|▎         | 17/485 [00:01<00:47,  9.86it/s, now=None]





t:   4%|▎         | 18/485 [00:01<00:47,  9.75it/s, now=None]





t:   4%|▍         | 19/485 [00:01<00:48,  9.69it/s, now=None]





t:   4%|▍         | 20/485 [00:01<00:48,  9.49it/s, now=None]





t:   4%|▍         | 21/485 [00:02<00:49,  9.47it/s, now=None]





t:   5%|▍         | 22/485 [00:02<00:48,  9.54it/s, now=None]





t:   5%|▍         | 23/485 [00:02<00:48,  9.52it/s, now=None

TypeError: expected non-empty vector for x







t:  27%|██▋       | 133/485 [00:29<00:33, 10.63it/s, now=None]

In [100]:
test_video_output = '../output_video/harder_challenge_video_output_fullImageSlidingWindows.mp4'
project_video = VideoFileClip('../harder_challenge_video.mp4')
processed_video = project_video.fl_image(pipeline)
%time processed_video.write_videofile(test_video_output, audio = False, verbose=False)
















                                                                    








t:   0%|          | 1/1260 [57:45<1211:54:08, 3465.33s/it, now=None]




t:   0%|          | 1/1260 [55:33<1165:51:36, 3333.67s/it, now=None]





t:   0%|          | 1/1260 [59:43<1253:17:57, 3583.70s/it, now=None]



t:   0%|          | 1/1260 [56:26<1184:25:09, 3386.74s/it, now=None]

t:   0%|          | 1/1260 [58:37<1230:15:50, 3517.83s/it, now=None]














                                                                    








t:   0%|          | 1/1260 [57:45<1211:54:28, 3465.34s/it, now=None]




t:   0%|          | 1/1260 [55:33<1165:51:55, 3333.69s/it, now=None]





t:   0%|          | 1/1260 [59:43<1253:18:15, 3583.71s/it, now=None]



t:   0%|          | 1/1260 [56:26<1184:25:26, 3386.76s/it, now=None]

t:   0%|          | 1/1260 [58:37<1230:16:07, 3517.85s/it, now=None]






t:   0%|          | 0/1199 [00:00<?, ?it/s, now=None]






t:   0%|          | 2/1199 [00:00<0

Moviepy - Building video ../output_video/harder_challenge_video_output_fullImageSlidingWindows.mp4.
Moviepy - Writing video ../output_video/harder_challenge_video_output_fullImageSlidingWindows.mp4










t:   0%|          | 3/1199 [00:00<01:37, 12.30it/s, now=None]






t:   0%|          | 4/1199 [00:00<01:47, 11.09it/s, now=None]






t:   0%|          | 5/1199 [00:00<01:54, 10.42it/s, now=None]






t:   1%|          | 6/1199 [00:00<01:59, 10.02it/s, now=None]






t:   1%|          | 7/1199 [00:00<02:01,  9.79it/s, now=None]






t:   1%|          | 8/1199 [00:00<02:05,  9.50it/s, now=None]






t:   1%|          | 9/1199 [00:00<02:11,  9.02it/s, now=None]






t:   1%|          | 10/1199 [00:01<02:14,  8.82it/s, now=None]






t:   1%|          | 11/1199 [00:01<02:14,  8.82it/s, now=None]






t:   1%|          | 12/1199 [00:01<02:14,  8.86it/s, now=None]






t:   1%|          | 13/1199 [00:01<02:15,  8.75it/s, now=None]






t:   1%|          | 14/1199 [00:01<02:16,  8.71it/s, now=None]






t:   1%|▏         | 15/1199 [00:01<02:16,  8.70it/s, now=None]






t:   1%|▏         | 16/1199 [00:01<02:17,  8.63it/s, now=None]






t:   1%|▏         | 17/1199 [00:01

Moviepy - Done !
Moviepy - video ready ../output_video/harder_challenge_video_output_fullImageSlidingWindows.mp4
CPU times: user 5min 9s, sys: 2min 27s, total: 7min 37s
Wall time: 2min 25s
